In [1]:
from func import *

In [7]:
from tqdm import tqdm
import re

path_to_reads = 'myreads.sam'

if path_to_reads[-3:] == 'sam':
    sequence = read_sam('myreads.sam')
elif path_to_reads[-5:] == 'fastq':
    sequence = read_fastq('myreads.fastq')
elif path_to_reads[-5:] == 'fasta':
    sequence = read_fasta('myreads.fasta')
write_seq_in_file_with_length('myreads.fasta', sequence, 0, 0)

# подсчёт средней длины последовательности, чтобы избавиться от слишком коротких и слишком длинных последовательностей
alligment = read_seq_from_file('myreads.fasta')
avg_length = sum(len(word) for word in alligment) / len(alligment)
list_allig = []
list_need_seq = []
for j, i in enumerate(alligment):
    if len(i) >= avg_length / 1.6 or len(i) <= avg_length / 0.66:
        list_allig.append(i)
        list_need_seq.append(j)
write_seq_in_file_with_length('myreads.fasta', list_allig, 0, 0)

# Стартовые позиции последовательности, у нас всегда 0
list_startpos = []
for i in list_allig:
    list_startpos.append(0)

# Узнавание какая последовательность самая длинная и её индекс
sequence = read_seq_from_file('myreads.fasta')
maxi_index = max([len(word) for word in sequence])
index_max = 0
for j, i in enumerate(sequence):
    if len(i) == maxi_index:
        index_max = j

# Основная часть, в которой сначала проверяется подходит ли нам последовательность учитывая стартовую позицию
# Далее отрезается часть последовательности от indi до indi + 80, где indi это индекс, который увеличивается на половину длины выравнивания 
# Далее считается количество пропусков и нуклеотидов и заменяется в изначальных последовательностях куски нуклеотидов на выравненные 
list_already_use = []
list_lish_gap = []
indi = 0
count_nucl = 0
for start in tqdm(range(0, 20000, 40)):
    list_use_seq = []
    list_use_seq_index = []
    

    for j, seq in enumerate(sequence):
        if count_nucl >= list_startpos[j]:
            if j not in list_already_use:
                sequence[j] = indi * '-' + seq + (10000 - len(seq) - list_startpos[j]) * '-'
                list_already_use.append(j)
            gap_flag = False
            for i in seq[indi:indi + 80]:
                if i != '-':
                    gap_flag = True
                    continue
            if gap_flag:
                list_use_seq.append(sequence[j])
                list_use_seq_index.append(j)    

    write_seq_in_file_with_length_and_name(f'allig/reads_{indi}-{indi + 80}.fasta', list_use_seq, list_use_seq_index, indi, indi + 80)
    muscle(f'allig/reads_{indi}-{indi + 80}.fasta', f'allig/allig_{indi}-{indi + 80}.fasta')

    list_seq_with_insert = []
    sorted_seqs = sort_fasta(f'allig/allig_{indi}-{indi + 80}.fasta')
    for rec in sorted_seqs:
        list_seq_with_insert.append(str(rec.seq))
    write_seq_in_file_with_length_and_name(f'allig/allig_{indi}-{indi + 80}.fasta', list_seq_with_insert, list_use_seq_index,  0, 0)

    alligment = read_seq_from_file(f'allig/allig_{indi}-{indi + 80}.fasta')



    tmp = int(0.5 * len(alligment[0]))

    list_allig_index = []
    list_count_gap = []
    for j, seq in enumerate(alligment):
        tmp2 = seq[:tmp].count('-')
        list_count_gap.append(tmp2)
        list_allig_index.append(tmp - tmp2)
    
    for i in range(len(alligment)):
        alligment[i] = alligment[i][:tmp]

    for j, i in enumerate(list_use_seq_index):
        if i == index_max:
            count_nucl += int(list_allig_index[j] * 0.93)
        sequence[i] = sequence[i][:indi] + alligment[j] + sequence[i][list_allig_index[j] + indi:]
    indi += tmp

# Дополнение строк до одинаковой длины
list_length = []
for i in sequence:
    list_length.append(len(i))
maxi = max(list_length)
for j, i in enumerate(sequence):
    while len(sequence[j]) < maxi:
        sequence[j] = sequence[j] + '-'

write_seq_in_file_with_length('allig_seq.fasta', sequence, 0, 0)

# Постройка консенсуса 
list_consensus = []
alligment = read_seq_from_file('allig_seq.fasta')
count_nucl = per_nucl_in_cal(alligment)
count_gap = count_gaps (alligment)
alligment = delete_gap(alligment, count_gap, count_nucl, 50 / 100)
# Запись множественного выравнивания в файл
write_seq_in_file_with_length('multiple_alignment.fasta', alligment, 0, 0)
pi = start_probability (alligment)
a = matrix_of_MDI(alligment)
A = probability_of_trans(alligment, a)
count = count_nucl_in_cal(alligment)
b = percetn_of_nucl_in_cal(count)
consensuss = consensus(alligment, b)
# Запись консенсуса в файл
write_seq_in_file_with_length('consensus.fasta', [consensuss], 0, 0)
# Расчёт качества
consi = read_seq_from_file('consensus.fasta')
multiple_allig = read_seq_from_file('multiple_alignment.fasta')
list_seq = consi + multiple_allig
write_seq_in_file_with_length('consensus_and_multiple.fasta', list_seq, 0, 0)
list_prob_quality = quality(list_seq)
phred_string = convert_to_phred(list_prob_quality)
write_fastq('consensus.fastq', consi[0][:-100], phred_string)


100%|██████████| 500/500 [00:06<00:00, 76.94it/s] 
